# 17C: Gender Segmentation (Male)

Model 17C segments the market by gender for males. (pp. 135)

Import larch and verify that the version number is correct (as of 1/30/2024 should be 6.0.17)

In [ ]:
import larch

In [ ]:
larch.__version__

This example is a mode choice model built using the MTC example dataset. First we create the DB and Model objects:

In [ ]:
d = larch.examples.MTC(format="dataset")
d

In [ ]:
d = d.sel(caseid=d.femdum == 0)

In [ ]:
m = larch.Model(d, compute_engine="numba")

Then we can build up the utility function. We’ll use some :ref:idco data first, using the Model.utility.co attribute. This attribute is a dict-like object, to which we can assign :class:LinearFunction objects for each alternative code.

In [ ]:
from larch import P, X

for a in [4, 5, 6]:
    m.utility_co[a] += X("hhinc") * P(f"hhinc#{a}")

Since the model we want to create groups together DA, SR2 and SR3+ jointly as reference alternatives with respect to income, we can simply omit all of these alternatives from the block that applies to hhinc.

For vehicles per worker, the preferred model include a joint parameter on SR2 and SR3+, but not including DA and not fixed at zero. Here we might use a shadow_parameter (also called an alias in some places), which allows us to specify one or more parameters that are simply a fixed proportion of another parameter. For example, we can say that vehbywrk_SR2 will be equal to vehbywrk_SR.

In [ ]:
for i in d["alt_names"][1:3]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        +X("vehbywrk") * P("vehbywrk_SR")
        + X("wkccbd+wknccbd") * P("wkcbd_" + name)
        + X("wkempden") * P("wkempden_" + name)
        + P("ASC_" + name)
    )

for i in d["alt_names"][3:]:
    name = str(i.values)
    a = int(i.altid)
    m.utility_co[a] += (
        +X("vehbywrk") * P("vehbywrk_" + name)
        + X("wkccbd+wknccbd") * P("wkcbd_" + name)
        + X("wkempden") * P("wkempden_" + name)
        + P("ASC_" + name)
    )

Next we’ll use some idca data, with the utility_ca attribute. This attribute is only a single :class:LinearFunction that is applied across all alternatives using :ref:idca data. Because the data is structured to vary across alternatives, the parameters (and thus the structure of the :class:LinearFunction) does not need to vary across alternatives.

In [ ]:
m.utility_ca = (
    +X("totcost/hhinc") * P("costbyincome")
    + X("tottime * (altid <= 4)") * P("motorized_time")
    + X("tottime * (altid >= 5)") * P("nonmotorized_time")
    + X("ovtt/dist * (altid <= 4)") * P("motorized_ovtbydist")
)

The “totcost/hhinc” data is computed once as a new variable when loading the model data. The same applies for tottime filtered by motorized modes (we harness the convenient fact that all the motorized modes have identifying numbers 4 or less), and “ovtt/dist”.

Lastly, we need to identify idca Format data that gives the availability for each alternative, as well as the number of times each alternative is chosen. (In traditional discrete choice analysis, this is often 0 or 1, but it need not be binary, or even integral.)


In [ ]:
m.availability_ca_var = "avail"
m.choice_ca_var = "chose"

And let’s give our model a descriptive title.

In [ ]:
m.title = "MTC Example 17C, Segmented for males"

We can view a summary of the choices and alternative availabilities to make sure the model is set up correctly.

In [ ]:
m.choice_avail_summary()

We’ll set a parameter cap (bound) at +/- 25, which helps improve the numerical stability of the optimization algorithm used in estimation.

In [ ]:
m.set_cap(25)

Having created this model, we can then estimate it:

In [ ]:
assert m.compute_engine == "numba"

In [ ]:
result = m.maximize_loglike(stderr=True, options={"maxiter": 1000, "ftol": 1e-10})
m.calculate_parameter_covariance()
m.loglike()

In [ ]:
m.parameter_summary()

It is a little tough to read this report because the parameters show up in alphabetical order. We can use the reorder method to fix this and group them systematically:

In [ ]:
m.ordering = (
    (
        "LOS",
        ".*cost.*",
        ".*time.*",
        ".*dist.*",
    ),
    (
        "Zonal",
        "wkcbd.*",
        "wkempden.*",
    ),
    (
        "Household",
        "hhinc.*",
        "vehbywrk.*",
    ),
    (
        "ASCs",
        "ASC.*",
    ),
)

In [ ]:
m.parameter_summary()

Finally, let's print model statistics.  Note that if you want LL at constants you need to run a separate model.

In [ ]:
m.estimation_statistics()

In [ ]:
# TEST
revealed_x = dict(zip(m.pnames, result.x))

In [ ]:
# TEST
from pytest import approx

expected_x = {
    "ASC_Bike": -1.928007825476385,
    "ASC_SR2": -1.912455790177904,
    "ASC_SR3+": -3.5514203840398006,
    "ASC_Transit": -0.8658094170164065,
    "ASC_Walk": -1.2203008866494844,
    "costbyincome": -0.06390233682221723,
    "hhinc#4": -0.0020804355665526397,
    "hhinc#5": -0.0013784543619515673,
    "hhinc#6": -0.00501951838450115,
    "motorized_ovtbydist": -0.18639734021256957,
    "motorized_time": -0.01951022691589421,
    "nonmotorized_time": -0.024446076816243063,
    "vehbywrk_Bike": -0.9930978890574966,
    "vehbywrk_SR": -0.20981196496890442,
    "vehbywrk_Transit": -0.8322463141471179,
    "vehbywrk_Walk": -0.6105805888289391,
    "wkcbd_Bike": 0.3204533731833982,
    "wkcbd_SR2": 0.027661592161988033,
    "wkcbd_SR3+": 1.423466120894847,
    "wkcbd_Transit": 1.196921606447681,
    "wkcbd_Walk": 0.22281387280820592,
    "wkempden_Bike": 0.0004958705725495997,
    "wkempden_SR2": 0.0009305263934769419,
    "wkempden_SR3+": 0.0005787415453590467,
    "wkempden_Transit": 0.002487830750207477,
    "wkempden_Walk": 0.0012923574145590863,
}
for k in expected_x:
    assert revealed_x[k] == approx(expected_x[k], 2e-2), (
        f"{k}, {revealed_x[k] / expected_x[k]}"
    )